In [5]:
from pyowm import OWM
import requests
import random
import re
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
import geocoder

def getLink(link, display_text):
    response = requests.get(link)
    return f'<a href="{response.url}" target="_blank">{display_text}</a>'


def getTemp():

    owm = OWM('9b86683eb9873777046595f152f8f21e')

    g = geocoder.ip('me')
    loc = g.city + "," + g.country
    mgr = owm.weather_manager()
    observation = mgr.weather_at_place(loc)
    weather = observation.weather

    temp = weather.temperature('fahrenheit')
    return "It currently feels like " + str(temp['feels_like']) + " degrees fahrenheit, but it's actually " + str(temp['temp']) + " degrees fahrenheit right now."

def ai_request(question):

    url = "https://chatgpt-best-price.p.rapidapi.com/v1/chat/completions"

    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": "a41195d370msh7d4b5a40c6dffe7p1cab4cjsn01da0f9acd93",
        "X-RapidAPI-Host": "chatgpt-best-price.p.rapidapi.com"
    }

    response = requests.post(url, json=payload, headers=headers)
    json_response = response.json()
    return json_response['choices'][0]['message']['content']

app = Flask(__name__)
CORS(app)
 

# Dictionary of bot responses based on user input patterns
bot_responses = {
    r"hi|hello|hey": ["Hello!", "Hi there!", "Hey!"],
    r"how are you.*": ["I'm doing well, thank you!", "I'm great! How about you?"],
    r"what.*your name": ["You can call me ChatBot.", "I'm ChatBot, nice to meet you!"],
    r"bye|goodbye": ["Goodbye! Have a great day!", "Farewell! Take care!"],
    r"tell me a joke": ["Why don't scientists trust atoms? Because they make up everything!"],
    r"were you created|were you made" : ["I was created on June 8, 2023", "I was created in June"],
    r"can you.*math": ["I currently cannot do math, however here is the link to a reputable website: " + getLink("https://tinyurl.com/calculatorHelpDesk", "Link")],
    r"weather" : [getTemp()],
    r"customer service" : ["Thank you! I am connecting you with a customer service agent"],
    r"about aditi" : ["Aditi is a leading consultancy that connects tech talent, tech teams, and project solutions to help our clients accelerate their digital journey. If you would like to know more about us, please click the following link: " + getLink("https://www.aditiconsulting.com/culture", "Link")]
    # r".*": ["I'm sorry, I can't help with that. However, click the following link to chat with a live customer service agent: " + getLink("https://teams.live.com/l/invite/FAAm0hfHKYim8bjiwE", "Link")]     
}

 

# Function to select a random response from a list
def get_random_response(responses):
    return random.choice(responses)

 

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json['user_input']
    matched_response = None
    for pattern, responses in bot_responses.items():
        if re.search(pattern, user_input, re.IGNORECASE):
            matched_response = get_random_response(responses)
            break

    # If no predefined responses match, make a request to the AI API
    if matched_response is None:
        try:
            matched_response = ai_request(user_input)
        except Exception as e:
            # If the API request fails, provide a generic response
            matched_response = "I'm sorry, I can't help with that. However, click the following link to chat with a live customer service agent: " + getLink("https://teams.live.com/l/invite/FAAm0hfHKYim8bjiwE", "Link")

    return jsonify({'response': matched_response})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Jul/2023 13:21:33] "OPTIONS /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:21:34] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:23:41] "OPTIONS /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:23:43] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:24:02] "OPTIONS /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:24:03] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:24:10] "OPTIONS /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:24:10] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:24:54] "OPTIONS /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:24:54] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:25:05] "OPTIONS /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 13:25:06] "POST /chatbot HTTP/1.1" 200 -
